In [ ]:
%run -i 'gradCAM.py' --image_path "dataset/PlantVillage/PlantVillage-Dataset/raw/color/Corn_(maize)___Common_rust_/RS_Rust 1609.JPG" --model_path models/plant_village_VGG19_trained_01.h5

In [ ]:
# !apt update && apt install -y libsm6 libxext6
# !apt install -y libxrender1
# !pip install tf-explain
# https://tf-explain.readthedocs.io
# !pip install pillow

In [1]:
import tensorflow as tf
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input, decode_predictions

# from tf_explain.core.grad_cam import GradCAM

In [2]:
model_1 = tf.keras.models.load_model('models/plant_village_VGG19_trained_01.h5')

In [3]:
model = tf.keras.models.Sequential()
for lay in model_1.layers[0].layers :
    model.add(lay)
model.add(model_1.layers[1])
model.add(model_1.layers[2])

In [4]:
grad_model = tf.keras.models.Model(
            [model.inputs], [model.get_layer('block5_conv4').output, model.output]
        )

In [5]:
img = tf.keras.preprocessing.image.load_img('dataset/PlantVillage/PlantVillage-Dataset/raw/color/Corn_(maize)___Common_rust_/RS_Rust 1609.JPG', 
                    target_size= (224,224)
                    )

In [6]:
x = image.img_to_array(img)
x = preprocess_input(x)
data = np.expand_dims(x, 0)
# data = ([x], None)

In [9]:
pred = model.predict(data)
class_index = np.argmax(pred)

In [60]:
num_classes = 27
expected_output = tf.one_hot([class_index] * data.shape[0], num_classes)

In [61]:
with tf.GradientTape() as tape:
    tape.watch(conv_outputs)
    inputs = tf.cast(data, tf.float32)
    conv_outputs, predictions = grad_model(inputs)
    loss = tf.keras.losses.categorical_crossentropy(expected_output, predictions)
    # loss = predictions[:, class_index]

In [62]:
    grads = tape.gradient(loss, conv_outputs)

In [63]:
grads == None

True

In [48]:
# guided_grads = (tf.cast(conv_outputs > 0, "float32") * tf.cast(grads > 0, "float32") * grads)
# Real one - Dont work because cant compute tape.gradient(loss, conv_outputs), returns NoneType
guided_grads = tf.cast(conv_outputs > 0, "float32") 

In [49]:
def ponderate_output(output, grad):
        """
        Perform the ponderation of filters output with respect to average of gradients values.
        Args:
            output (tf.Tensor): Target layer outputs, with shape (Hl, Wl, Nf),
                where Hl and Wl are the target layer output height and width, and Nf the
                number of filters.
            grads (tf.Tensor): Guided gradients with shape (Hl, Wl, Nf)
        Returns:
            tf.Tensor: Ponderated output of shape (Hl, Wl, 1)
        """
        weights = tf.reduce_mean(grad, axis=(0, 1))

        # Perform ponderated sum : w_i * output[:, :, i]
        cam = tf.reduce_sum(tf.multiply(weights, output), axis=-1)

        return cam

In [51]:
maps = [ponderate_output(output, grad) for output, grad in zip(conv_outputs, guided_grads)]

In [ ]:
heatmaps = np.array([heatmap_display(cam.numpy(), image, colormap) for cam, image in zip(maps, data)])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [41]:
class_index = 26
explainer = GradCAM()

In [42]:
grid = explainer.explain(data, model, 'block5_conv4', class_index)

TypeError: in converted code:

    /usr/local/lib/python3.6/dist-packages/tf_explain/core/grad_cam.py:88 get_gradients_and_filters  *
        guided_grads = (

    TypeError: '>' not supported between instances of 'NoneType' and 'int'


In [57]:
explainer.save(grid, '.', 'grad_cam3.png')